In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import joblib
import matplotlib.pyplot as plt
    # """Loads and preprocesses the dataset."""
df = pd.read_csv("D:\\All Data till 21st Feb 2025\\D-MART ANALYSIS\\fd_tickets_202502181901.csv")
df.columns = df.columns.str.strip()

In [7]:
df["status"].value_counts()


status
resolved    95274
closed      15128
open         3450
Resolved      942
Closed        657
pending        51
New             2
Name: count, dtype: int64

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import joblib
import matplotlib.pyplot as plt

def load_and_preprocess_data(file_path):
    """Loads and preprocesses the dataset."""
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    
    # Convert date columns to datetime
    date_columns = ["First Trx Date", "Last Trx Date", "created_at"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Normalize status labels
    df["status"] = df["status"].str.lower()
    
    # Drop columns with excessive missing values
    drop_cols = ["Chain", "Store Address", "Software Version", "First Trx Date", "Last Trx Date"]
    df.drop(columns=drop_cols, inplace=True, errors='ignore')
    
    # Handle missing values
    df.fillna("Unknown", inplace=True)
    
    # Encode categorical variables before resampling
    label_encoders = {}
    categorical_cols = ["Store", "State", "City", "Pin Code", "Terminal Category", "issue_type", "cf_issue_sub_category", "status"]
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    # Convert all columns to numeric explicitly
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Fill missing values with median for numerical columns
    df.fillna(df.median(), inplace=True)
    
    # Define target variable (predicting 'open' vs. 'closed')
    df["Target"] = df["status"].apply(lambda x: 1 if x == "open" else 0)
    
    # Check if dataset contains only one class
    if df["Target"].nunique() < 2:
        print("Error: Only one class present in dataset. Cannot train model.")
        print("Suggestion: Ensure the dataset contains both 'open' and 'closed' statuses.")
        return None, None, None
    
    return df, label_encoders, df["Target"].nunique()

def train_model(df):
    """Trains a RandomForest model with stratified cross-validation and undersampling to balance classes."""
    if df is None:
        print("Skipping training due to insufficient data.")
        return None, None
    
    X = df.drop(columns=["Target"])
    y = df["Target"]
    
    # Ensure class balance in splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Apply Random Undersampling to balance classes
    rus = RandomUnderSampler(random_state=42)
    X_train, y_train = rus.fit_resample(X_train, y_train)
    print(f"Training class distribution after undersampling: {np.bincount(y_train)}")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = RandomForestClassifier(
        n_estimators=100, 
        max_depth=10, 
        min_samples_split=10, 
        min_samples_leaf=5, 
        max_features='sqrt', 
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    
    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=skf, scoring='roc_auc')
    print(f"Cross-Validation ROC-AUC: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
    
    # Save the model and scaler
    joblib.dump(model, "dmart_issue_predictor.pkl")
    joblib.dump(scaler, "scaler.pkl")
    
    print("Model training complete.")
    return model, scaler

if __name__ == "__main__":
    # Load and preprocess the data
    file_path = "fd_tickets_202502181901.csv"
    df, label_encoders, class_count = load_and_preprocess_data(file_path)
    
    if df is None:
        exit()
    
    # Train the model
    model, scaler = train_model(df)
    if model is None:
        exit()
    
    # Example prediction (for the first row of the dataset)
    sample_data = df.drop(columns=["Target"]).iloc[:1]
    sample_data = sample_data.apply(pd.to_numeric, errors='coerce')
    sample_data.fillna(sample_data.median(), inplace=True)
    
    if model and scaler:
        predicted_issue = predict_issues(model, scaler, sample_data)
        issue_type = label_encoders["issue_type"].inverse_transform([predicted_issue[0]])[0]
        print(f"Predicted Issue Type: {issue_type}")
    else:
        print("Skipping prediction due to failed training.")


C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_15560\1591448785.py:19: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_15560\1591448785.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)


Error: Only one class present in dataset. Cannot train model.
Suggestion: Ensure the dataset contains both 'open' and 'closed' statuses.
Skipping training due to insufficient data.


AttributeError: 'NoneType' object has no attribute 'drop'

: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import joblib
import matplotlib.pyplot as plt

def load_and_preprocess_data(file_path):
    """Loads and preprocesses the dataset."""
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    
    # Convert date columns to datetime
    date_columns = ["First Trx Date", "Last Trx Date", "created_at"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Normalize status labels
    df["status"] = df["status"].str.lower()
    
    # Drop columns with excessive missing values
    drop_cols = ["Chain", "Store Address", "Software Version", "First Trx Date", "Last Trx Date"]
    df.drop(columns=drop_cols, inplace=True, errors='ignore')
    
    # Handle missing values
    df.fillna("Unknown", inplace=True)
    
    # Encode categorical variables before resampling
    label_encoders = {}
    categorical_cols = ["Store", "State", "City", "Pin Code", "Terminal Category", "issue_type", "cf_issue_sub_category", "status"]
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    # Convert all columns to numeric explicitly
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Fill missing values with median for numerical columns
    df.fillna(df.median(), inplace=True)
    
    # Define target variable (predicting 'open' vs. 'closed')
    df["Target"] = df["status"].apply(lambda x: 1 if x == "open" else 0)
    
    # Check if dataset contains only one class
    if df["Target"].nunique() < 2:
        print("Error: Only one class present in dataset. Cannot train model.")
        print("Suggestion: Ensure the dataset contains both 'open' and 'closed' statuses.")
        return None, None, None
    
    return df, label_encoders, df["Target"].nunique()

def train_model(df):
    """Trains a RandomForest model with stratified cross-validation and undersampling to balance classes."""
    if df is None:
        print("Skipping training due to insufficient data.")
        return None, None
    
    X = df.drop(columns=["Target"])
    y = df["Target"]
    
    # Ensure class balance in splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    from imblearn.under_sampling import RandomUnderSampler
    rus = RandomUnderSampler(random_state=42)
    X_train, y_train = rus.fit_resample(X_train, y_train)

    

    
#     # Apply Random Undersampling to balance classes
#     rus = RandomUnderSampler(random_state=42)
#     X_train, y_train = rus.fit_resample(X_train, y_train)
#     print(f"Training class distribution after undersampling: {np.bincount(y_train)}")
    
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
    
#     model = RandomForestClassifier(
#         n_estimators=100, 
#         max_depth=10, 
#         min_samples_split=10, 
#         min_samples_leaf=5, 
#         max_features='sqrt', 
#         random_state=42
#     )
#     model.fit(X_train_scaled, y_train)
    
#     # Cross-validation
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=skf, scoring='roc_auc')
#     print(f"Cross-Validation ROC-AUC: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
    
#     # Save the model and scaler
#     joblib.dump(model, "dmart_issue_predictor.pkl")
#     joblib.dump(scaler, "scaler.pkl")
    
#     print("Model training complete.")
#     return model, scaler

# if __name__ == "__main__":
#     # Load and preprocess the data
#     file_path = "fd_tickets_202502181901.csv"
#     df, label_encoders, class_count = load_and_preprocess_data(file_path)
    
#     if df is None:
#         exit()
    
#     # Train the model
#     model, scaler = train_model(df)
#     if model is None:
#         exit()
    
#     # Example prediction (for the first row of the dataset)
#     sample_data = df.drop(columns=["Target"]).iloc[:1]
#     sample_data = sample_data.apply(pd.to_numeric, errors='coerce')
#     sample_data.fillna(sample_data.median(), inplace=True)
    
#     # if model and scaler:
#         predicted_issue = predict_issues(model, scaler, sample_data)
#         issue_type = label_encoders["issue_type"].inverse_transform([predicted_issue[0]])[0]
#         print(f"Predicted Issue Type: {issue_type}")
#     else:
#         print("Skipping prediction due to failed training.")


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import joblib
import matplotlib.pyplot as plt

def load_and_preprocess_data(file_path):
    """Loads and preprocesses the dataset."""
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    
    # Convert date columns to datetime
    date_columns = ["First Trx Date", "Last Trx Date", "created_at"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Normalize status labels
    df["status"] = df["status"].str.lower()
    
    # Drop columns with excessive missing values
    drop_cols = ["Chain", "Store Address", "Software Version", "First Trx Date", "Last Trx Date"]
    df.drop(columns=drop_cols, inplace=True, errors='ignore')
    
    # Handle missing values
    df.fillna("Unknown", inplace=True)
    
    # Encode categorical variables before resampling
    label_encoders = {}
    categorical_cols = ["Store", "State", "City", "Pin Code", "Terminal Category", "issue_type", "cf_issue_sub_category", "status"]
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    # Convert all columns to numeric explicitly
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Fill missing values with median for numerical columns
    df.fillna(df.median(), inplace=True)
    
    # Define target variable (predicting 'open' vs. 'closed')
    # df["Target"] = df["status"].apply(lambda x: 1 if x == "open" else 0)
    # df["Target"] = df["status"].apply(lambda x: 1 if "open" in x.lower() else 0)
    df["status"] = df["status"].astype(str)  # Convert entire column to string first
    df["Target"] = df["status"].apply(lambda x: 1 if "open" in x.lower() else 0)


    # Check if dataset contains only one class
    if df["Target"].nunique() < 2:
        print("Error: Only one class present in dataset. Cannot train model.")
        print("Suggestion: Ensure the dataset contains both 'open' and 'closed' statuses.")
        return None, None, None
    
    return df, label_encoders, df["Target"].nunique()

def train_model(df):
    """Trains a RandomForest model with stratified cross-validation and undersampling to balance classes."""
    if df is None:
        print("Skipping training due to insufficient data.")
        return None, None
    
    X = df.drop(columns=["Target"])
    y = df["Target"]
    
    # Ensure class balance in splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Apply Random Undersampling to balance classes
    # rus = RandomUnderSampler(random_state=42)
    # rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
    # rus = RandomUnderSampler(sampling_strategy={0: 2 * df["Target"].sum(), 1: df["Target"].sum()}, random_state=42)

    from imblearn.over_sampling import RandomOverSampler
    ros = RandomOverSampler(sampling_strategy=0.5, random_state=42)
    X_train, y_train = ros.fit_resample(X_train, y_train)



    X_train, y_train = rus.fit_resample(X_train, y_train)
    print(f"Training class distribution after undersampling: {np.bincount(y_train)}")
    
    if df is None or model is None:
     print("Skipping prediction due to insufficient data.")
    exit()


    

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = RandomForestClassifier(
        n_estimators=100, 
        max_depth=10, 
        min_samples_split=10, 
        min_samples_leaf=5, 
        max_features='sqrt', 
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    
    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=skf, scoring='roc_auc')
    print(f"Cross-Validation ROC-AUC: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
    
    # Save the model and scaler
    joblib.dump(model, "dmart_issue_predictor.pkl")
    joblib.dump(scaler, "scaler.pkl")
    
    print("Model training complete.")
    return model, scaler

if __name__ == "__main__":
    # Load and preprocess the data
    file_path = "fd_tickets_202502181901.csv"
    df, label_encoders, class_count = load_and_preprocess_data(file_path)
    print("Class distribution after preprocessing:")
    print(df["Target"].value_counts())

    
    # if df is None:
    #     exit()
    if df is None:
        print("Error: Dataset is empty or contains only one class. Exiting.")
        exit()

    # Train the model
    model, scaler = train_model(df)
    if model is None:
        exit()
    
    # Example prediction (for the first row of the dataset)
    sample_data = df.drop(columns=["Target"]).iloc[:1]
    sample_data = sample_data.apply(pd.to_numeric, errors='coerce')
    sample_data.fillna(sample_data.median(), inplace=True)
    
    if model and scaler:
        predicted_issue = predict_issues(model, scaler, sample_data)
        issue_type = label_encoders["issue_type"].inverse_transform([predicted_issue[0]])[0]
        print(f"Predicted Issue Type: {issue_type}")
    else:
        print("Skipping prediction due to failed training.")


C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\3544930658.py:19: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\3544930658.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)


Error: Only one class present in dataset. Cannot train model.
Suggestion: Ensure the dataset contains both 'open' and 'closed' statuses.
Class distribution after preprocessing:


TypeError: 'NoneType' object is not subscriptable

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
import joblib
import matplotlib.pyplot as plt

def load_and_preprocess_data(file_path):
    """Loads and preprocesses the dataset without using the status column."""
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    
    # Convert date columns to datetime
    date_columns = ["First Trx Date", "Last Trx Date", "created_at"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Drop unnecessary columns
    drop_cols = ["Chain", "Store Address", "Software Version", "First Trx Date", "Last Trx Date", "status"]
    df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True, errors='ignore')
    
    # Handle missing values
    df.fillna("Unknown", inplace=True)
    
    # Encode categorical variables
    label_encoders = {}
    categorical_cols = ["Store", "State", "City", "Pin Code", "Terminal Category", "issue_type", "cf_issue_sub_category"]
    for col in categorical_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            label_encoders[col] = le
    
    # Convert all columns to numeric explicitly
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Fill missing values with median for numerical columns
    df.fillna(df.median(), inplace=True)
    
    # Define target variable (predicting issue frequency per store)
    df["Target"] = df.groupby("Store")["ticket_id"].transform("count")
    
    return df, label_encoders

def train_model(df):
    """Trains a RandomForest model to predict issue frequency per store."""
    X = df.drop(columns=["Target"])
    y = df["Target"]
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = RandomForestClassifier(
        n_estimators=100, 
        max_depth=10, 
        min_samples_split=10, 
        min_samples_leaf=5, 
        max_features='sqrt', 
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    
    # Cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=skf, scoring='r2')
    print(f"Cross-Validation R2 Score: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
    
    # Save the model and scaler
    joblib.dump(model, "dmart_issue_predictor.pkl")
    joblib.dump(scaler, "scaler.pkl")
    
    print("Model training complete.")
    return model, scaler

if __name__ == "__main__":
    # Load and preprocess the data
    file_path = "fd_tickets_202502181901.csv"
    df, label_encoders = load_and_preprocess_data(file_path)
    
    # Train the model
    model, scaler = train_model(df)
    
    # Example prediction (for the first row of the dataset)
    sample_data = df.drop(columns=["Target"]).iloc[:1]
    sample_data = sample_data.apply(pd.to_numeric, errors='coerce')
    sample_data.fillna(sample_data.median(), inplace=True)
    
    if model and scaler:
        predicted_issue_count = model.predict(scaler.transform(sample_data))[0]
        print(f"Predicted Number of Issues for Store: {predicted_issue_count}")


C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\153348212.py:18: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\153348212.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)


Cross-Validation R2 Score: 0.8624 (+/- 0.0119)
Model training complete.
Predicted Number of Issues for Store: 265


In [12]:
pip install uvicorn

   ---------------------------------------- 0.0/62.3 kB ? eta -:--:--
   -------------------------------- ------- 51.2/62.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 62.3/62.3 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
   ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\akanksha.yadav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from fastapi import FastAPI
import uvicorn

# Load and preprocess the dataset
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    
    # Convert date columns to datetime
    date_columns = ["First Trx Date", "Last Trx Date", "created_at"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Drop unnecessary columns
    drop_cols = ["Chain", "Store Address", "Software Version", "First Trx Date", "Last Trx Date", "status"]
    df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True, errors='ignore')
    
    # Handle missing values
    df.fillna("Unknown", inplace=True)
    
    # Encode categorical variables
    label_encoders = {}
    categorical_cols = ["Store", "State", "City", "Pin Code", "Terminal Category", "issue_type", "cf_issue_sub_category"]
    for col in categorical_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            label_encoders[col] = le
    
    # Convert all columns to numeric explicitly
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Fill missing values with median for numerical columns
    df.fillna(df.median(), inplace=True)
    
    # Define target variable (predicting issue frequency per store)
    df["Target"] = df.groupby("Store")["ticket_id"].transform("count")
    
    return df, label_encoders

# Train the model
def train_model(df):
    X = df.drop(columns=["Target"])
    y = df["Target"]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = RandomForestClassifier(
        n_estimators=100, 
        max_depth=10, 
        min_samples_split=10, 
        min_samples_leaf=5, 
        max_features='sqrt', 
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    
    # Save the model and scaler
    joblib.dump(model, "dmart_issue_predictor.pkl")
    joblib.dump(scaler, "scaler.pkl")
    
    print("Model training complete.")
    return model, scaler

# Load trained model
try:
    model = joblib.load("dmart_issue_predictor.pkl")
    scaler = joblib.load("scaler.pkl")
except:
    model, scaler = None, None

# FastAPI chatbot application
app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Welcome to the D-Mart Operations Chatbot! Ask anything about D-Mart issues."}

@app.get("/query")
def query_data(query: str):
    if "highest issues" in query.lower():
        top_store = df.groupby("Store")["Target"].sum().idxmax()
        return {"response": f"The store with the highest number of issues is {top_store}."}
    elif "issues in last month" in query.lower():
        last_month = df[df["created_at"] >= pd.to_datetime("now") - pd.DateOffset(months=1)]
        return {"response": f"There were {len(last_month)} issues reported in the last month."}
    elif "store" in query.lower():
        store_name = query.split("store")[-1].strip()
        store_issues = df[df["Store"] == store_name]
        return {"response": f"Store {store_name} has {len(store_issues)} reported issues."}
    else:
        return {"response": "I couldn't understand your question. Please ask about D-Mart operations!"}

@app.get("/predict")
def predict_issues(store_id: int):
    if model is None or scaler is None:
        return {"error": "Model not trained yet!"}
    
    store_data = df[df["Store"] == store_id].drop(columns=["Target"]).iloc[:1]
    if store_data.empty:
        return {"error": "Store ID not found in dataset!"}
    
    store_data = store_data.apply(pd.to_numeric, errors='coerce')
    store_data.fillna(store_data.median(), inplace=True)
    predicted_issues = model.predict(scaler.transform(store_data))[0]
    
    return {"store_id": store_id, "predicted_issues": int(predicted_issues)}

if __name__ == "__main__":
    file_path = "fd_tickets_202502181901.csv"
    df, label_encoders = load_and_preprocess_data(file_path)
    if model is None:
        model, scaler = train_model(df)
    
    uvicorn.run(app, host="0.0.0.0", port=8000)


C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\1555256078.py:19: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\1555256078.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)


RuntimeError: asyncio.run() cannot be called from a running event loop

In [16]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from fastapi import FastAPI
import uvicorn

# Load and preprocess the dataset
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    
    # Convert date columns to datetime
    date_columns = ["First Trx Date", "Last Trx Date", "created_at"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Drop unnecessary columns
    drop_cols = ["Chain", "Store Address", "Software Version", "First Trx Date", "Last Trx Date", "status"]
    df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True, errors='ignore')
    
    # Handle missing values
    df.fillna("Unknown", inplace=True)
    
    # Encode categorical variables
    label_encoders = {}
    categorical_cols = ["Store", "State", "City", "Pin Code", "Terminal Category", "issue_type", "cf_issue_sub_category"]
    for col in categorical_cols:
        if col in df.columns:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            label_encoders[col] = le
    
    # Convert all columns to numeric explicitly
    df = df.apply(pd.to_numeric, errors='coerce')
    
    # Fill missing values with median for numerical columns
    df.fillna(df.median(), inplace=True)
    
    # Define target variable (predicting issue frequency per store)
    df["Target"] = df.groupby("Store")["ticket_id"].transform("count")
    
    return df, label_encoders

# Train the model
def train_model(df):
    X = df.drop(columns=["Target"])
    y = df["Target"]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = RandomForestClassifier(
        n_estimators=100, 
        max_depth=10, 
        min_samples_split=10, 
        min_samples_leaf=5, 
        max_features='sqrt', 
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    
    # Save the model and scaler
    joblib.dump(model, "dmart_issue_predictor.pkl")
    joblib.dump(scaler, "scaler.pkl")
    
    print("Model training complete.")
    return model, scaler

# Load trained model
try:
    model = joblib.load("dmart_issue_predictor.pkl")
    scaler = joblib.load("scaler.pkl")
except:
    model, scaler = None, None

# FastAPI chatbot application
app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Welcome to the D-Mart Operations Chatbot! Ask anything about D-Mart issues."}

@app.get("/query")
def query_data(query: str):
    if "highest issues" in query.lower():
        top_store = df.groupby("Store")["Target"].sum().idxmax()
        return {"response": f"The store with the highest number of issues is {top_store}."}
    elif "issues in last month" in query.lower():
        last_month = df[df["created_at"] >= pd.to_datetime("now") - pd.DateOffset(months=1)]
        return {"response": f"There were {len(last_month)} issues reported in the last month."}
    elif "store" in query.lower():
        store_name = query.split("store")[-1].strip()
        store_issues = df[df["Store"] == store_name]
        return {"response": f"Store {store_name} has {len(store_issues)} reported issues."}
    else:
        return {"response": "I couldn't understand your question. Please ask about D-Mart operations!"}

@app.get("/predict")
def predict_issues(store_id: int):
    if model is None or scaler is None:
        return {"error": "Model not trained yet!"}
    
    store_data = df[df["Store"] == store_id].drop(columns=["Target"]).iloc[:1]
    if store_data.empty:
        return {"error": "Store ID not found in dataset!"}
    
    store_data = store_data.apply(pd.to_numeric, errors='coerce')
    store_data.fillna(store_data.median(), inplace=True)
    predicted_issues = model.predict(scaler.transform(store_data))[0]
    
    return {"store_id": store_id, "predicted_issues": int(predicted_issues)}

# if __name__ == "__main__":
#     file_path = "fd_tickets_202502181901.csv"
#     df, label_encoders = load_and_preprocess_data(file_path)
#     if model is None:
#         model, scaler = train_model(df)
    
#     uvicorn.run(app, host="0.0.0.0", port=8000)


import asyncio

if __name__ == "__main__":
    file_path = "fd_tickets_202502181901.csv"
    df, label_encoders = load_and_preprocess_data(file_path)
    if model is None:
        model, scaler = train_model(df)
    
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)

    if asyncio.get_event_loop().is_running():
        print("Running inside an active event loop. Use `!uvicorn your_script:app --reload` in Jupyter Notebook or VS Code terminal.")
    else:
        server.run()


C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\1475795404.py:19: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_63692\1475795404.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)


Running inside an active event loop. Use `!uvicorn your_script:app --reload` in Jupyter Notebook or VS Code terminal.


In [1]:
pip install spacy


   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.2 MB 5.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.4/12.2 MB 5.8 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/12.2 MB 6.1 MB/s eta 0:00:02
   ---------------- ----------------------- 5.0/12.2 MB 6.0 MB/s eta 0:00:02
   ------------------- -------------------- 6.0/12.2 MB 5.9 MB/s eta 0:00:02
   ----------------------- ---------------- 7.1/12.2 MB 5.7 MB/s eta 0:00:01
   -------------------------- ------------- 8.1/12.2 MB 5.5 MB/s eta 0:00:01
   ----------------------------- ---------- 8.9/12.2 MB 5.4 MB/s eta 0:00:01
   -------------------------------- ------- 10.0/12.2 MB 5.2 MB/s eta 0:00:01
   ----------------------------------- ---- 10.7/12.2 MB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.2 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.2 MB 4.9 MB/s eta 0:00:01
 

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")
print("SpaCy model loaded successfully!")


SpaCy model loaded successfully!
